In [5]:
# Dependencies
import numpy as np
from sklearn.datasets import load_iris
from graphviz import Digraph
from sklearn import tree
import matplotlib.pyplot as plt
import random
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold

## Implementation
### Splitting

In [6]:
def train_test_split(data, test_size=0.2, random_state=2060):
    """
    Split the data into training and testing sets randomly.

    Parameters:
    - data: 2D numpy array, the entire dataset.
    - test_size: float, the proportion of the data to include in the test split.
    - random_state: int, the seed used by the random number generator.

    Returns:
    - train_data: 2D numpy array, the training set.
    - test_data: 2D numpy array, the testing set.
    """
    if random_state is not None:
        np.random.seed(random_state)
    n_samples = data.shape[0]
    indices = np.random.permutation(n_samples) # shuffling
    test_size = int(n_samples * test_size)
    test_indices = indices[:test_size]
    train_indices = indices[test_size:]
    train_data = data[train_indices]
    test_data = data[test_indices]
    return train_data, test_data

### CART

In [9]:
class Node:
    '''
    Constructor for the Node class
    '''
    def __init__(self, left=None, right=None, label=None, feature=None, threshold=None, parent_gini=None, node_gini=None, num_samples=None, class_counts=None):
        self.left = left # to a left node
        self.right = right # to a right node
        self.label = label
        self.feature = feature
        self.threshold = threshold
        self.parent_gini = parent_gini
        self.node_gini = node_gini
        self.num_samples = num_samples # data points in this node
        self.class_counts = class_counts

    def is_leaf(self):
        '''
        check if the node is a leaf node
        '''
        return self.label is not None

In [10]:
class CART:
    '''
    Decision Tree classifier by UC Providence
    '''
    def __init__(self, max_depth=None, min_samples_split=2, ccp_alpha=0.01, random_state=0):
        if max_depth is None:
            self.max_depth = 20
        else:
            self.max_depth = max_depth # set the max depth
        self.min_samples_split = min_samples_split
        self.tree = None
        self.ccp_alpha = ccp_alpha # for pruning
        self.random_state = random_state
        if random_state is not None: # make random state deterministic
            np.random.seed(random_state)
            random.seed(random_state)
        
    def fit(self, data):
        '''
        build the tree based on the data
        '''
        self.tree = self._build_tree(data, depth=0)
    
    def prune(self, ccp_alpha=0):
        '''
        prune the tree based on the ccp_alpha
        '''
        self._prune_tree(self.tree, ccp_alpha)
    
    def predict(self, data):
        '''
        Helper function to predict the data
        '''
        X = data[:, :-1] # get the features
        return np.array([self._predict_row(self.tree, row) for row in X])

    def loss(self, data):
        '''
        Helper function to calculate the loss
        '''
        preds = self.predict(data)
        true_labels = data[:, -1] # last column is the label
        return np.sum(preds != true_labels) / len(true_labels)

    def accuracy(self, data):
        '''
        Helper function to calculate the accuracy
        '''
        return 1 - self.loss(data)
    
    def _gini_for_node(self, data):
        '''
        Get the gini index for a node
        params data: the data in the node
        return: the gini index
        '''
        labels = data[:, -1] # get the last column which is the label
        _, counts = np.unique(labels, return_counts=True)
        probs = counts / len(labels)
        parent_gini = 1 - np.sum(probs ** 2) # calculate the gini index
        return parent_gini

    def _gini_for_split(self, data, left, right):
        '''
        Get the gini index for a split
        params data: the data in the node
        params left: the left split
        params right: the right split
        return: the gini index
        '''
        # calc the total size
        total_size = len(data)
        left_size = len(left)
        right_size = len(right)
        # calc the gini index for the left and right
        left_gini = self._gini_for_node(left)
        right_gini = self._gini_for_node(right)
        # calc the weighted gini index
        weighted_gini = (left_size / total_size) * left_gini + (right_size / total_size) * right_gini
        return weighted_gini

    def _split(self, data, feature_index, threshold):
        '''
        Split the data based on the feature and threshold
        params data: the data
        params feature_index: the feature to split on
        params threshold: the threshold to split on
        return: the left and right split
        '''
        left = data[data[:, feature_index] <= threshold]
        right = data[data[:, feature_index] > threshold]
        return left, right

    def _find_best_split(self, data):
        '''
        Find the best split for the data, traverse through each column and each average value of the values in the column to find the best split.
        params data: the dataset
        return: the best gain and the best split
        '''
        best_gain = float("-inf")
        best_split = None
        best_split_list = [] # for ties
        parent_gini = self._gini_for_node(data) # calc the gini index for the parent node
        n_features = data.shape[1] - 1
        for feature in range(n_features): # traverse through each feature
            unique_values = np.unique(data[:, feature])
            sorted_values = np.sort(unique_values)
            thresholds = (sorted_values[1:] + sorted_values[:-1]) / 2 # get the average of the values

            if len(thresholds) > 2:
                # Continuous or ordinal features
                for threshold in thresholds:
                    left, right = self._split(data, feature, threshold)
                    if len(left) == 0 or len(right) == 0:
                        continue # skip if the split is empty
                    weighted_gini = self._gini_for_split(data, left, right) # calc the weighted gini index
                    gain = parent_gini - weighted_gini
                    if gain > best_gain: # if the gain is better than the best gain
                        best_gain = gain
                        best_split_list = [{
                            "feature": feature,
                            "threshold": threshold,
                            "gini_for_split": weighted_gini,
                            "parent_gini": parent_gini,
                            "gain": gain,
                            "left": left,
                            "right": right,
                            "type": "continuous"
                        }]
                    elif np.isclose(gain, best_gain):    
                        best_gain = gain
                        best_split_list.append({
                            "feature": feature,
                            "threshold": threshold,
                            "gini_for_split": weighted_gini,
                            "parent_gini": parent_gini,
                            "gain": gain,
                            "left": left,
                            "right": right,
                            "type": "continuous"
                        }) # if tied, then append to the list
            else:
                # Only one threshold for binary features
                for threshold in thresholds:
                    left, right = self._split(data, feature, threshold)
                    if len(left) == 0 or len(right) == 0:
                        continue
                    weighted_gini = self._gini_for_split(data, left, right)
                    gain = parent_gini - weighted_gini
                    if gain > best_gain:    
                        best_gain = gain
                        # same for binary features but with different type
                        best_split_list = [{
                            "feature": feature,
                            "threshold": threshold,
                            "gini_for_split": weighted_gini,
                            "parent_gini": parent_gini,
                            "gain": gain,
                            "left": left,
                            "right": right,
                            "type": "binary"
                        }]
                    elif np.isclose(gain, best_gain):    
                        best_gain = gain
                        best_split_list.append({
                            "feature": feature,
                            "threshold": threshold,
                            "gini_for_split": weighted_gini,
                            "parent_gini": parent_gini,
                            "gain": gain,
                            "left": left,
                            "right": right,
                            "type": "binary"
                        })
        if len(best_split_list) > 1:
            # if the best split list has more than one best split then we sort it by feature
            #print("Multiple best splits found")
            #x = sorted(best_split_list, key=lambda x: x["feature"])
            # for i in x:
            #     print(i["feature"], i["threshold"], i["gini_for_split"], i["parent_gini"], i["gain"])
            best_split = sorted(best_split_list, key=lambda x: x["feature"])[-1]
        else: # else we just get the first best split
            best_split = best_split_list[0]
        return best_gain, best_split

    def _majority_class(self, data):
        '''
        Get the majority class in the data
        '''
        labels = data[:, -1] # get the labels
        unique_labels, counts = np.unique(labels, return_counts=True)
        return unique_labels[np.argmax(counts)]

    def _build_tree(self, data, depth=0):
        '''
        Build the tree recursively
        params data: the data
        params depth: the depth of the tree
        return: the node and its attributes
        '''
        labels = data[:, -1] # label is the last column
        num_samples = len(labels)
        parent_gini = self._gini_for_node(data)

        # Stopping conditions
        # Having a pure node
        if len(np.unique(labels)) == 1:
            return Node(label=labels[0], parent_gini=parent_gini, num_samples=num_samples)
        # Max depth reached
        if self.max_depth is not None and depth >= self.max_depth:
            return Node(label=self._majority_class(data), parent_gini=parent_gini, num_samples=num_samples)
        # Minimum samples split reached
        if num_samples < self.min_samples_split:
            return Node(label=self._majority_class(data), parent_gini=parent_gini, num_samples=num_samples)
        # No split found
        best_gain, best_split = self._find_best_split(data)
        if best_gain == 0:
            return Node(label=self._majority_class(data), parent_gini=parent_gini, num_samples=num_samples)

        # Put left and right data into the tree
        if best_split["type"] == "binary":
            remaining_left = best_split["left"]
            remaining_right = best_split["right"]
        else:
            remaining_left = best_split["left"]
            remaining_right = best_split["right"]
        # Recursion
        left_tree = self._build_tree(remaining_left, depth + 1)
        right_tree = self._build_tree(remaining_right, depth + 1)
        return Node(
            left=left_tree,
            right=right_tree,
            feature=best_split["feature"],
            threshold=best_split["threshold"], 
            parent_gini=parent_gini,
            num_samples=num_samples
        )

    def _predict_row(self, node, row):
        '''
        recursively predict the row
        params node: the node
        params row: the row
        return: the prediction
        '''
        if node.is_leaf():
            return node.label
        else:
            if row[node.feature] <= node.threshold:
                return self._predict_row(node.left, row)
            else:
                return self._predict_row(node.right, row)
    
    def _count_leaves(self, node):
        '''Helper function to count the number of leaves in a subtree'''
        if node.is_leaf():
            return 1
        else:
            return self._count_leaves(node.left) + self._count_leaves(node.right)

## Unit Tests

In [11]:
# Tests for splitting
def test_splitting():
    # Check if the splitted sizes are correct
    data = np.array([[i] for i in range(101)])
    train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)
    assert train_data.shape[0] == 71, "Train set size is incorrect."
    assert test_data.shape[0] == 30, "Test set size is incorrect."
    # Check if the function works well for empty data
    data = np.array([]).reshape(0, 2)
    train_data, test_data = train_test_split(data, test_size=0.4, random_state=42)
    assert train_data.shape[0] == 0, "Train set should be empty."
    assert test_data.shape[0] == 0, "Test set should be empty."
    print("Splitting tests passed.")

# Tests for Node
def test_Node():
    # Check if the node is a leaf node
    leaf_node = Node(label=1)
    assert leaf_node.is_leaf(), "Leaf node should be a leaf."
    # Check if the node is a decision node
    decision_node = Node(left="LeftNode", right="RightNode", feature=2, threshold=0.5)
    assert not decision_node.is_leaf(), "Decision node should not be a leaf."
    print("Node tests passed.")

# Tests for _find_best_split
def test_find_best_split():
    data = np.array([
        [1.0, 2.5, 0],
        [2.0, 3.5, 1],
        [1.5, 2.0, 0]
    ])
    cart = CART()
    best_split = cart._find_best_split(data)
    # Check a valid split is found and the split is correct
    # Should not split on the target but split on one of the continuous features
    assert best_split is not None, "Best split should not be None."
    assert best_split["type"] == "continuous", "Best split should be continuous."
    assert best_split["feature"] != 2, "Best split should not be on the target."
    data = np.array([
        [1.0, 2.5, 0],
        [1.0, 2.5, 0],
        [1.0, 2.5, 0]
    ])
    cart = CART()
    best_split = cart._find_best_split(data)
    # Check that no split should be found if all features are the same
    assert best_split is None, "Best split should be None."
    print("Find best split tests passed.")

# Tests for Gini impurity calculations
def test_gini():
    data = np.array([
        [1.0, 2.5, 0],
        [2.0, 3.5, 1]
    ])
    cart = CART()
    gini = cart._gini_for_node(data) # Expected = 0.5
    assert abs(gini - 0.5) < 1e-6, "Gini impurity for node is incorrect."
    left = data[:1]
    right = data[1:]
    gini = cart._gini_for_split(data, left, right) # Expected = 0
    assert abs(gini - 0) < 1e-6, "Gini impurity for split is incorrect."
    print("Gini tests passed.")

# Tests for splitting continuous features and categorical features
def test_split_features():
    data = np.array([
        [1.0, 2.5, 0],
        [2.0, 3.5, 1],
        [1.0, 2.0, 0],
        [2.0, 4.5, 1]
    ])
    cart = CART()
    left, right = cart._split_continuous(data, 1, 3.0)
    # Check if the split is correct for a continuous feature
    assert len(left) == 2, "Left split size is incorrect."
    assert len(right) == 2, "Right split size is incorrect."
    left, right = cart._split_categorical(data, 2)
    # Check if the split is correct for a categorical feature
    assert len(left) == 2, "Left split size is incorrect."
    assert len(right) == 2, "Right split size is incorrect."
    print("Split features tests passed.")

# Tests for fit and _build_tree
def test_fit():
    data = np.array([
        [1, 2, 0],
        [3, 4, 1],
        [1, 2, 0],
        [3, 4, 1]
    ])
    cart = CART(max_depth=2, min_samples_split=2)
    cart.fit(data)
    # Check if the tree is built
    assert cart.tree is not None, "Tree should not be None after fitting."
    assert cart.tree.feature is not None, "Tree root should have a splitting feature."
    data = np.empty((0, 3))
    cart.fit(data)
    # Check if the label is 0
    assert cart.tree.label == 0, "Incorrect label for empty data."
    print("Fit tests passed.")

# Tests for predict
def test_predict():
    # Check if a single row prediction is correct
    tree = Node(left=Node(label=1), right=Node(label=0), feature=0, threshold=1.5)
    cart = CART()
    row = np.array([1.0, 2.0])  # Expected: left -> 1
    pred = cart._predict_row(tree, row)
    assert pred == 1, "Prediction for single row is incorrect."
    # Check predictions for a dataset
    cart.tree = tree
    data = np.array([
        [1.0, 2.0],
        [2.0, 3.0]
    ])
    preds = cart.predict(data)
    assert np.array_equal(preds, [1, 0]), "Batch predictions are incorrect."
    print("Predict tests passed.")

# Tests for loss and accuracy
def test_loss_acc():
    tree = Node(left=Node(label=1), right=Node(label=0), feature=0, threshold=1.5)
    cart = CART()
    cart.tree = tree
    data = np.array([
        [1.0, 2.0, 1],
        [2.0, 3.0, 0],
        [0.5, 1.0, 1],
        [3.0, 4.0, 0]
    ])
    # Check if loss = 0 and accuracy = 1
    assert cart.loss(data) == 0.0, "Loss calculation is incorrect."
    assert cart.accuracy(data) == 1.0, "Accuracy calculation is incorrect."
    print("Loss and accuracy tests passed.")

In [15]:
test_splitting()
test_Node()
# test_find_best_split()
test_gini()
# test_split_features()
# test_fit()
test_predict()
test_loss_acc()

Splitting tests passed.
Node tests passed.
Gini tests passed.
Predict tests passed.
Loss and accuracy tests passed.


## Main

In [7]:
data = np.loadtxt("../data/heart.csv", delimiter=",", skiprows=1)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=2060)

model = CART(max_depth=10, min_samples_split=10)
model.fit(train_data)
train_accuracy = model.accuracy(train_data)
test_accuracy = model.accuracy(test_data)
print(f"Training accuracy: {train_accuracy:.3f}")
print(f"Testing accuracy: {test_accuracy:.3f}")
model.visualize()

Training accuracy: 0.901
Testing accuracy: 0.893
--- START PRINT TREE ---
Split attribute = 2; threshold = 0.500
Left:
  Split attribute = 11; threshold = 0.500
  Left:
    Split attribute = 12; threshold = 2.500
    Left:
      Split attribute = 8; categorical
      Left:
        Split attribute = 7; threshold = 96.500
        Left:
          Predict -> 0.0
        Right:
          Split attribute = 4; threshold = 316.500
          Left:
            Predict -> 1.0
          Right:
            Predict -> 0
      Right:
        Split attribute = 6; threshold = 0.500
        Left:
          Predict -> 1.0
        Right:
          Split attribute = 9; threshold = 1.500
          Left:
            Split attribute = 3; threshold = 115.000
            Left:
              Predict -> 1.0
            Right:
              Predict -> 0.0
          Right:
            Predict -> 1.0
    Right:
      Split attribute = 9; threshold = 0.650
      Left:
        Split attribute = 0; threshold = 42.000
 

## Scikit-Learn Verification

Below the CART model using scikit-learning is written by our group members, Yixun Kang and David Ning. The model use a pipeline structure, including the preprocessor and the algorithm. In the preprocessor, we used One-Hot encoder for two categorical features, `sex` and `exange` and then applied `StandardScaler()` to all features. We used K-Fold as the cross validation and 6/2/2 for train/test/validation. In the parameter grid, we tuned for `min_samples_leaf` and `max_leaf_nodes` and keep the `max_depth` and `min_samples_split` the same as in main.

In [8]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, ParameterGrid
from sklearn.pipeline import Pipeline, make_pipeline
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [9]:
data = pd.read_csv("../data/heart.csv")
X = data.drop(columns=["target"])
y = data["target"]

In [10]:
# Preprocessor
cat_ftrs = ["sex", "exang"]
num_ftrs = ["age", "trestbps", "chol", "thalach", "oldpeak", "cp", "fbs", "restecg", "slope", "ca", "thal"]

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(sparse_output=False, handle_unknown="ignore")),
    ("scaler", StandardScaler())
])
numerical_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])
preprocessor = ColumnTransformer([
    ("cat", categorical_transformer, cat_ftrs),
    ("num", numerical_transformer, num_ftrs)
])

In [11]:
# ML pipeline
def MLpipe_kfold(X, y, random_states, preprocessor, ML_algo, param_grid, n_splits=5):
    test_scores = []
    best_models = []
    for i, random_state in enumerate(random_states):
        X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
        pipe = make_pipeline(preprocessor, ML_algo)
        grid = GridSearchCV(pipe, param_grid=param_grid, cv=kf, n_jobs=-1, return_train_score=True, 
                            verbose=True, scoring="accuracy")
        grid.fit(X_other, y_other)
        results = pd.DataFrame(grid.cv_results_)
        best_models.append(grid)
        y_test_pred = best_models[-1].predict(X_test)
        test_accuracy = accuracy_score(y_test, y_test_pred)
        test_scores.append(test_accuracy)
    return test_scores, best_models

In [12]:
random_states = [2060]
ML_algo = DecisionTreeClassifier(random_state=2060, criterion="gini", max_depth=10, min_samples_split=10)
param_grid = {
    "decisiontreeclassifier__min_samples_leaf": [1, 2, 5, 10],
    "decisiontreeclassifier__max_leaf_nodes": [5, 10]
}
test_scores, best_models = MLpipe_kfold(X, y, random_states, preprocessor, ML_algo, param_grid, n_splits=10)
print("Average Testing Accuracy:", np.mean(test_scores))

Fitting 10 folds for each of 8 candidates, totalling 80 fits
Average Testing Accuracy: 0.8829268292682927
